In [1]:
import pandas as pd
import numpy as np

In [2]:
df_name = pd.read_excel('../data/생관3-612-24-004_2024년 3월 일일생산계획 및 업체별 차종현황_Rev.01_24.03.06(확정).xlsx', skiprows=[0, 1, 2], usecols='C', nrows=64)
df_data = pd.read_excel('../data/생관3-612-24-004_2024년 3월 일일생산계획 및 업체별 차종현황_Rev.01_24.03.06(확정).xlsx', skiprows=[0, 1, 2, 3], usecols='F:BE', nrows=64)
df_data2 = pd.read_excel('../data/생관3-612-24-007_2024년 4월 일일생산계획 및 업체별 차종현황_Rev.00_24.03.28(확정).xlsx', skiprows=[0, 1, 2, 3], usecols='F:BE', nrows=64)

In [3]:
df_name = df_name.drop(index=df_name.index[0])
df_name = df_name.reset_index(drop=True)
df_name = df_name.ffill()
df_data = df_data.drop(index=df_data.index[-1])
df_data2 = df_data2.drop(index=df_data2.index[-1])
df_data.index = df_name['세부공정명'].tolist()
df_data2.index = df_name['세부공정명'].tolist()

In [4]:
def clean_column_names(df):
    columns = df.columns.tolist()
    for i in range(1, len(columns)):  # 첫 번째 열은 처리할 필요 없음
        if isinstance(columns[i], str) and "Unnamed" in columns[i]:
            columns[i] = columns[i - 1]  # 바로 왼쪽 열 이름으로 대체
    df.columns = columns
    return df

# DataFrame 열 이름 정리
df_data = clean_column_names(df_data)
df_data2 = clean_column_names(df_data2)

In [5]:
class SubwayCar:
    def __init__(self, name):
        self.name = name
        self.activity_dict = dict()
    
    def add_activity(self, activity_name, date):
        if activity_name not in self.activity_dict:
            self.activity_dict[activity_name] = date
        else:
            print(f"subwaycar {self.name} activity {activity_name}: {date} is already taken")

In [6]:
def create_subway_car_dict(df_data, subway_car_dict, start_date):
    for col_idx in range(df_data.shape[1]):
            date = df_data.columns[col_idx]  # 현재 열의 이름
    
            for row_idx in range(df_data.shape[0]):  # 행 반복
                activity_name = df_data.index[row_idx]  # 현재 행 이름
                car_name = df_data.iloc[row_idx, col_idx]  # 값 가져오기
    
                if pd.notna(car_name):  # NaN 값 제외
                    if car_name not in subway_car_dict:
                        subway_car_dict[car_name] = SubwayCar(name=car_name)
                    
                    subway_car_dict[car_name].add_activity(activity_name, start_date + pd.Timedelta(days=date - 1))
    return subway_car_dict

In [7]:
subway_car_dict = {}  # {name: SubwayCar 객체}

subway_car_dict = create_subway_car_dict(df_data, subway_car_dict, pd.to_datetime('2024-03-01'))
print('----------------')
print('----------------')
print('----------------')
print('----------------')
print('----------------')
subway_car_dict = create_subway_car_dict(df_data2, subway_car_dict, pd.to_datetime('2024-04-01'))



----------------
----------------
----------------
----------------
----------------
subwaycar S035-M2 activity  실내/상하 CABLE HARNESS 취부: 2024-04-01 00:00:00 is already taken
subwaycar S021-TC1 activity  실내 배선 작업: 2024-04-01 00:00:00 is already taken
subwaycar S012-M1 activity  배전반 취부 및 배선작업: 2024-04-01 00:00:00 is already taken
subwaycar S035-M2 activity  실내/상하 배선: 2024-04-02 00:00:00 is already taken
subwaycar S022-M1 activity  실내 배선 작업: 2024-04-02 00:00:00 is already taken
subwaycar S011-TC1 activity  CAB MODULE 취부(TC): 2024-04-02 00:00:00 is already taken
subwaycar S013-M2 activity  배전반 취부 및 배선작업: 2024-04-02 00:00:00 is already taken
subwaycar S011-TC1 activity  COUPLER 취부: 2024-04-02 00:00:00 is already taken
subwaycar S020-TC2 activity  상하 전장기기 취부 결선: 2024-04-02 00:00:00 is already taken
subwaycar S030-TC2 activity  실내 배선 작업: 2024-04-03 00:00:00 is already taken
subwaycar S040-TC2 activity  실내 배선 검사: 2024-04-03 00:00:00 is already taken
subwaycar S015-M2 activity  배전반 취부 및 배선작업: 2

In [8]:
# 결과 확인
for car_name, subway_car in subway_car_dict.items():
    print(f"SubwayCar Name: {car_name}")
    print("Activities:", subway_car.activity_dict)

SubwayCar Name: S025-M2
Activities: {' 측창취부/T-BOLT 삽입': Timestamp('2024-03-01 00:00:00'), ' 실내/상하 CABLE HARNESS 취부': Timestamp('2024-03-04 00:00:00'), ' 실내/상하 배선': Timestamp('2024-03-06 00:00:00'), ' 실내 배선 작업': Timestamp('2024-03-07 00:00:00'), ' 하부덕트검사\n (D+1~4일차 조정작업)': Timestamp('2024-03-08 00:00:00'), ' 실내 배선 검사': Timestamp('2024-03-09 00:00:00'), ' AIR DUCT MODULE 취부': Timestamp('2024-04-08 00:00:00'), ' CENTER GRILL 취부': Timestamp('2024-04-09 00:00:00'), ' 객실도어 취부 검사': Timestamp('2024-04-10 00:00:00'), ' AIR DUCT MODULE 취부 검사': Timestamp('2024-04-10 00:00:00'), ' 배관 MODULE 취부': Timestamp('2024-04-11 00:00:00'), ' 배전반 취부 및 배선작업': Timestamp('2024-04-12 00:00:00'), ' CENTER PIVOT 취부': Timestamp('2024-04-12 00:00:00'), ' COUPLER 취부': Timestamp('2024-04-13 00:00:00'), ' 제동기기 취부 및 누설검사': Timestamp('2024-04-13 00:00:00'), ' D+5~13일차 조정작업': Timestamp('2024-04-15 00:00:00'), ' ROOF 내장판 취부': Timestamp('2024-04-16 00:00:00'), ' 상하 전장기기 취부 결선': Timestamp('2024-04-17 00:00:00'), ' SIDE 내장판 취부